## ДЗ_8

### Выяснить какая архитектура больше подходит для задачи сентимент анализа на данных - CNN или RNN 

*1. Построить свёрточную архитектуру*

*2. Построить различные архитектуры с RNN*

*3. Построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)*

*4. Сделать выводы что получилось лучше*

**Загрузим необходимые библиотеки и скачаем данные.**

In [1]:
import pandas as pd
import numpy as np

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import keras
import tensorflow as tf
import tensorflow_addons as tfa
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard 

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

2022-11-16 21:10:53.232665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
df_val = pd.read_csv("./val.csv")

In [3]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [4]:
df_test.head()

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."


In [5]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].progress_apply(preprocess_text)
df_val['text'] = df_val['text'].progress_apply(preprocess_text)
df_test['text'] = df_test['text'].progress_apply(preprocess_text)

100%|████████████████████████████████████████████████████████████████████████████| 22684/22684 [00:30<00:00, 741.22it/s]


In [7]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [8]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [9]:
len(tokenizer.index_word)

258107

In [10]:
sequences_train[:10]

[[66107, 66108, 4712, 582, 3, 596],
 [1, 28151, 214, 6915, 451, 173, 9, 39004],
 [1, 1467, 1875, 654, 571, 1],
 [1, 66109, 41, 5700, 39005, 28152, 11483],
 [66110, 18433, 66111, 66112, 148, 362],
 [12632, 36, 123, 5, 24],
 [1, 66113, 15886, 7619, 22180, 67, 2150, 95, 3481, 27, 2150, 12633, 856],
 [169, 368, 3002, 66114, 3644, 113, 22181, 66115],
 [66116, 8571, 1185, 89],
 [12634, 4713, 2689, 39006]]

In [11]:
word_count, training_length

(258108, 27)

In [12]:
y_train = df_train['class'].values
y_val = df_val['class'].values

**Чтобы определить какая их архитектур нейронных сетей лучше справляется с поставленной задачей, построим несложную CNN-сеть и на ее основе проведем эксперименты - будем добавлять слои с другими сетями, при этом оставляя параметры сетей максимально похожими. Чтобы происходило соревнование алгоритмов.**

### CNN

In [13]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Conv1D(64, 3))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 30)            7743240   
                                                                 
 masking (Masking)           (None, 27, 30)            0         
                                                                 
 conv1d (Conv1D)             (None, 25, 128)           11648     
                                                                 
 dropout (Dropout)           (None, 25, 128)           0         
                                                                 
 activation (Activation)     (None, 25, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 64)            24640     
                                                                 
 dropout_1 (Dropout)         (None, 23, 64)            0

2022-11-16 21:15:49.443150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                                                 
Total params: 7,780,189
Trainable params: 7,780,189
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [15]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3) 

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 83s 257ms/step - loss: 0.6927 - accuracy: 0.5342 - f1_score: 0.6734 - val_loss: 0.6925 - val_accuracy: 0.5203 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 290s 908ms/step - loss: 0.6916 - accuracy: 0.5510 - f1_score: 0.6734 - val_loss: 0.6917 - val_accuracy: 0.5274 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 44s 138ms/step - loss: 0.6905 - accuracy: 0.5592 - f1_score: 0.6734 - val_loss: 0.6909 - val_accuracy: 0.5357 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 45s 141ms/step - loss: 0.6891 - accuracy: 0.5675 - f1_score: 0.6734 - val_loss: 0.6897 - val_accuracy: 0.5451 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 56s 177ms/step - loss: 0.6870 - accuracy: 0.5782 - f1_score: 0.6734 - val_loss: 0.6880 - val_accuracy: 0.5513 - val_f1_score: 0.6710


In [17]:
score_cnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn[0])
print('Test accuracy:', score_cnn[1])

45/45 [==============================] - 0s 8ms/step - loss: 0.6894 - accuracy: 0.5412 - f1_score: 0.6709


Test score: 0.6893725991249084
Test accuracy: 0.5411982536315918


### RNN

In [18]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_1 (Masking)         (None, 27, 30)            0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               20352     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [20]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3) 

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 43s 129ms/step - loss: 0.6933 - accuracy: 0.4973 - f1_score: 0.6734 - val_loss: 0.6919 - val_accuracy: 0.5139 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 43s 135ms/step - loss: 0.6913 - accuracy: 0.5099 - f1_score: 0.6734 - val_loss: 0.6897 - val_accuracy: 0.5391 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 46s 145ms/step - loss: 0.6892 - accuracy: 0.5289 - f1_score: 0.6734 - val_loss: 0.6870 - val_accuracy: 0.5508 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 51s 161ms/step - loss: 0.6867 - accuracy: 0.5500 - f1_score: 0.6734 - val_loss: 0.6840 - val_accuracy: 0.5611 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 53s 166ms/step - loss: 0.6841 - accuracy: 0.5647 - f1_score: 0.6734 - val_loss: 0.6812 - val_accuracy: 0.5686 - val_f1_score: 0.6710


In [21]:
score_rnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_rnn[0])
print('Test accuracy:', score_rnn[1])

45/45 [==============================] - 1s 21ms/step - loss: 0.6944 - accuracy: 0.5409 - f1_score: 0.6709


Test score: 0.6944053769111633
Test accuracy: 0.5408896803855896


### GRU

In [22]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(128,recurrent_dropout=0.2))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_2 (Masking)         (None, 27, 30)            0         
                                                                 
 gru (GRU)                   (None, 128)               61440     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 activation_8 (Activation)   (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)               

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [24]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3) 

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 142s 437ms/step - loss: 0.6931 - accuracy: 0.5169 - f1_score: 0.6734 - val_loss: 0.6928 - val_accuracy: 0.5422 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 291s 910ms/step - loss: 0.6923 - accuracy: 0.5554 - f1_score: 0.6734 - val_loss: 0.6912 - val_accuracy: 0.5715 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 77s 240ms/step - loss: 0.6888 - accuracy: 0.5919 - f1_score: 0.6734 - val_loss: 0.6851 - val_accuracy: 0.5984 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 240s 755ms/step - loss: 0.6784 - accuracy: 0.6141 - f1_score: 0.6734 - val_loss: 0.6706 - val_accuracy: 0.6119 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 131s 410ms/step - loss: 0.6575 - accuracy: 0.6367 - f1_score: 0.6734 - val_loss: 0.6471 - val_accuracy: 0.6339 - val_f1_score: 0.6710


In [25]:
score_gru = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_gru[0])
print('Test accuracy:', score_gru[1])

45/45 [==============================] - 2s 54ms/step - loss: 0.6626 - accuracy: 0.6108 - f1_score: 0.6709


Test score: 0.6626438498497009
Test accuracy: 0.610809862613678


### LSTM

In [26]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_3 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               81408     
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 activation_12 (Activation)  (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 64)               

In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [28]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3) 

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 162s 495ms/step - loss: 0.6929 - accuracy: 0.5216 - f1_score: 0.6734 - val_loss: 0.6923 - val_accuracy: 0.5634 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 155s 486ms/step - loss: 0.6913 - accuracy: 0.5738 - f1_score: 0.6734 - val_loss: 0.6894 - val_accuracy: 0.5978 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 115s 359ms/step - loss: 0.6859 - accuracy: 0.5974 - f1_score: 0.6734 - val_loss: 0.6802 - val_accuracy: 0.6038 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 156s 491ms/step - loss: 0.6712 - accuracy: 0.6080 - f1_score: 0.6734 - val_loss: 0.6607 - val_accuracy: 0.6116 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 534s 2s/step - loss: 0.6502 - accuracy: 0.6257 - f1_score: 0.6734 - val_loss: 0.6399 - val_accuracy: 0.6412 - val_f1_score: 0.6710


In [29]:
score_lstm = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_lstm[0])
print('Test accuracy:', score_lstm[1])

45/45 [==============================] - 2s 48ms/step - loss: 0.6550 - accuracy: 0.6090 - f1_score: 0.6709


Test score: 0.6550308465957642
Test accuracy: 0.6090023517608643


### CNN > RNN

In [30]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_4 (Masking)         (None, 27, 30)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 128)           11648     
                                                                 
 dropout_8 (Dropout)         (None, 25, 128)           0         
                                                                 
 activation_16 (Activation)  (None, 25, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_14 (Dense)            (None, 64)               

In [31]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [32]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3) 

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 69s 208ms/step - loss: 0.6929 - accuracy: 0.5261 - f1_score: 0.6734 - val_loss: 0.6924 - val_accuracy: 0.5639 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 70s 219ms/step - loss: 0.6916 - accuracy: 0.5590 - f1_score: 0.6734 - val_loss: 0.6907 - val_accuracy: 0.5750 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 78s 245ms/step - loss: 0.6887 - accuracy: 0.5778 - f1_score: 0.6734 - val_loss: 0.6863 - val_accuracy: 0.5868 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 77s 241ms/step - loss: 0.6816 - accuracy: 0.5895 - f1_score: 0.6734 - val_loss: 0.6769 - val_accuracy: 0.5903 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 85s 268ms/step - loss: 0.6693 - accuracy: 0.6004 - f1_score: 0.6734 - val_loss: 0.6643 - val_accuracy: 0.6028 - val_f1_score: 0.6710


In [33]:
score_cnn_rnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_cnn_rnn[0])
print('Test accuracy:', score_cnn_rnn[1])

45/45 [==============================] - 3s 63ms/step - loss: 0.6762 - accuracy: 0.5744 - f1_score: 0.6709


Test score: 0.6761654615402222
Test accuracy: 0.574394941329956


### RNN > CNN

In [34]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(128,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_5 (Masking)         (None, 27, 30)            0         
                                                                 
 lstm_2 (LSTM)               (None, 27, 128)           81408     
                                                                 
 conv1d_3 (Conv1D)           (None, 25, 128)           49280     
                                                                 
 dropout_10 (Dropout)        (None, 25, 128)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                      

In [35]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=1)])

In [36]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3) 

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
319/319 [==============================] - 140s 426ms/step - loss: 0.6926 - accuracy: 0.5073 - f1_score: 0.6734 - val_loss: 0.6924 - val_accuracy: 0.5049 - val_f1_score: 0.6710
Epoch 2/5
319/319 [==============================] - 141s 442ms/step - loss: 0.6910 - accuracy: 0.5321 - f1_score: 0.6734 - val_loss: 0.6907 - val_accuracy: 0.5231 - val_f1_score: 0.6710
Epoch 3/5
319/319 [==============================] - 355s 1s/step - loss: 0.6870 - accuracy: 0.5776 - f1_score: 0.6734 - val_loss: 0.6856 - val_accuracy: 0.5676 - val_f1_score: 0.6710
Epoch 4/5
319/319 [==============================] - 152s 473ms/step - loss: 0.6768 - accuracy: 0.5956 - f1_score: 0.6734 - val_loss: 0.6743 - val_accuracy: 0.6002 - val_f1_score: 0.6710
Epoch 5/5
319/319 [==============================] - 256s 802ms/step - loss: 0.6635 - accuracy: 0.6036 - f1_score: 0.6734 - val_loss: 0.6646 - val_accuracy: 0.6123 - val_f1_score: 0.6710


In [37]:
score_rnn_cnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score_rnn_cnn[0])
print('Test accuracy:', score_rnn_cnn[1])

45/45 [==============================] - 9s 195ms/step - loss: 0.6748 - accuracy: 0.5781 - f1_score: 0.6709


Test score: 0.6747561693191528
Test accuracy: 0.5781422257423401


**Для удобства выведем результаты в таблицу:**

In [38]:
result = pd.DataFrame({'model': ['CNN', 'RNN', 'GRU', 'LSTM','CNN_RNN', 'RNN_CNN'], 
                       'accuracy': [score_cnn[1], score_rnn[1], score_gru[1], 
                                    score_lstm[1], score_cnn_rnn[1], score_rnn_cnn[1]]})
result.sort_values(by='accuracy', ascending=False)

,model,accuracy
2,GRU,0.610810
3,LSTM,0.609002
5,RNN_CNN,0.578142
4,CNN_RNN,0.574395
0,CNN,0.541198
1,RNN,0.540890


#### ВЫВОД: результаты экспермента показывают, что при максимально равных условиях лучше всего справляются с поставленной задачей нейронные сети, способные бороться с "затуханием/взрывом" градиента (LSTM, GRU). Обе они лидируют с заметным отрывом и имеют практически одинаковый показатель "accuracy". На втором месте - совместные архитектуры (RNN_CNN, CNN_RNN). Они так же имеют очень похожий друг с другом показатель "accuracy", что подталкивает к выводу - не важно какой слой следует за каким. Результат получается одинаковым. И, неожиданно, на третьем месте оказались сети CNN и RNN. Их "accuracy" тоже практически одинаковый. Однако обычная CNN хоть и на тысячные доли, но обогнала сеть c рекуррентным слоем RNN! Можно только предположить, что набор данных был не очень большого размера и длинны. И затухание ингредиента не имело значимого влияния. В результате простая сеть с простой архитектурой показала чуть лучший результат.